In [ ]:
# ! pip install transformers datasets
# ! pip install evaluate
# ! pip install sentence-transformers

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. **Pre-training σε μεγάλα unlabelled datasets**:

  Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. **Fine-tuning σε μικρότερα labelled datasets**:
  
     Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του **text-classification pipeline** μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης.

Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες: **[entailment/neutral/contradiction]**.

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι **γραμματικά ορθό (acceptable) ή όχι (unacceptable)**:

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια.
Το  Yelp κατασκευάστηκε θεωρώντας τα αστέρια 1 και 2 αρνητικά και τα 3 και 4 θετικά.  Η αρνητική πολικότητα ανήκει στην κατηγορία 1 και η θετική στην κατηγορία 2. Τα reviews αυτά χωρίζονται σε αυτές τις κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.



In [ ]:
from datasets import load_dataset

# insert your code here

dataset = load_dataset("fancyzhx/yelp_polarity")

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [ ]:
# insert your code here

from collections import Counter

# Train dataset
train_dataset = dataset["train"].shuffle(seed=55).select(range(300))

# Test dataset
test_dataset = dataset["test"].shuffle(seed=55).select(range(300))

# Check the amount of label classes for each dataset
train_labels = [sample['label'] for sample in train_dataset]
test_labels = [sample['label'] for sample in test_dataset]
train_counts = Counter(train_labels)
test_counts = Counter(test_labels)

In [ ]:
train_counts, test_counts

In [ ]:
train_dataset

# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα.

Η διαδικασία αυτή επιτελείται μέσω των **Tokenizers**, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, **αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer**. Επιλέγοντας τη μέθοδο **from_pretrained** λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [ ]:
# insert your code here

from transformers import ElectraTokenizer, ElectraForSequenceClassification

tokenizer = ElectraTokenizer.from_pretrained("google/electra-base-discriminator")
model = ElectraForSequenceClassification.from_pretrained("google/electra-base-discriminator")

Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# insert your code here

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [ ]:
train_dataset

## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [ ]:
import numpy as np
import evaluate
import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε;  Σχολιάστε και αναλύστε.

In [ ]:
from transformers import TrainingArguments, Trainer, get_scheduler
from torch.optim import AdamW 



args = TrainingArguments(eval_strategy="epoch", per_device_train_batch_size=2, per_device_eval_batch_size = 2, num_train_epochs=5, learning_rate=3e-5, weight_decay=0.01,)


# insert your code here
# optimizer
optimizer = AdamW(model.parameters(), lr=args.learning_rate)
# scheduler
num_training_steps = len(train_dataset) // args.per_device_train_batch_size * args.num_train_epochs
scheduler = get_scheduler(
    name="linear",  # Linear decay scheduler
    optimizer=optimizer,
    num_warmup_steps=0,  # Warmup steps
    num_training_steps=num_training_steps,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

Στη συνέχεια, ρυθμίστε (fine-tune) το μοντέλο σας καλώντας το [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [ ]:
trained_model=trainer.train()

No. Run | Learning rate | Batch Size | Accuracy | Validation Loss
| - | - | - | - | - |
| 1 | 1e-5 | 12 | 0.913333 | 0.290150 |
| 2 | 3e-5 | 12 | 0.923333 | 0.274829 |
| 3 | 5e-5 | 12 | 0.926667 | 0.276333 |
| 4 | 7e-5 | 12 | 0.936667 | 0.276354 |
| 5 | 3e-5 | 8 | 0.920000 | 0.269310 |
| 6 | 3e-5 | 4 | 0.926667 | 0.332158 |
| 7 | 3e-5 | 2 | 0.923333 | 0.457162 |

- A moderate learning rate (3e-5 or 5e-5) provides the best trade-off between accuracy and validation loss. Higher learning rates (e.g., 7e-5) may lead to slightly better accuracy but at the cost of increased validation loss.
- Larger batch sizes (e.g., 12) provide more stable training and better validation loss. However, moderate batch sizes (e.g., 8) can sometimes improve generalization. Smaller batch sizes (e.g., 4 or 2) may lead to higher accuracy but at the cost of increased validation loss.
- The combination of a moderate learning rate (3e-5 or 5e-5) and a larger batch size (12) provides the best trade-off between accuracy and validation loss. However, if generalization is the priority, Run 5 (batch size 8) may be the best choice.

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks

Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή λογική (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [ ]:
# # insert your code here (load dataset)

from datasets import load_dataset

# insert your code here

dataset = load_dataset("ybisk/piqa", trust_remote_code=True)
dataset = dataset['train'].shuffle(seed=45).select(range(100))

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [ ]:
# insert your code here (models)

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMultipleChoice
import torch
from tqdm import tqdm


model_names = [
    "sledz08/finetuned-bert-piqa",
    "bert-base-uncased",
    "roberta-base",
    "albert-base-v2",
    "distilbert-base-uncased"
]

In [ ]:
# insert your code here (function for ending prediction)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_predictions(model_name, dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name).to(device)
    model.eval()
    correct = 0
    total = len(dataset)

    for sentence in tqdm(dataset):
        goal = sentence['goal']
        options = [sentence['sol1'], sentence['sol2']]
        label = sentence['label']
        
        encoding = tokenizer([goal] * len(options), options, return_tensors='pt', padding=True, truncation=True)
        input_ids = encoding["input_ids"].unsqueeze(0).to(device)
        attention_mask = encoding["attention_mask"].unsqueeze(0).to(device)

        if "token_type_ids" in encoding:
            token_type_ids = encoding["token_type_ids"].unsqueeze(0).to(device)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
        else:
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
        logits = outputs.logits  # Shape: (1, num_choices)
        prediction = torch.argmax(logits, dim=1).item()

        if prediction == label:
            correct += 1
        total += 1    
    accuracy = correct / total
    print(f"Accuracy for {model_name} (MultipleChoice): {accuracy:.4f}")
    return accuracy

results = {}
for model_name in model_names:
    try:
        accuracy = get_predictions(model_name, dataset)
        results[model_name] = accuracy
    except Exception as e:
        print(f"Skipping {model_name} due to error: {e}")

In [ ]:
results

## B2. Truthful QA

### Sentence Transformers

Οι **sentence transformers** χρησιμοποιούνται για να δημιουργήσουν **embeddings προτάσεων**, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

In [ ]:
model_names_semantic = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    "sentence-transformers/use-cmlm-multilingual",
    "intfloat/multilingual-e5-small",
    "sentence-transformers/paraphrase-MiniLM-L6-v2",
    "BAAI/bge-m3"
]

model_names_B1 = [
    "sledz08/finetuned-bert-piqa",
    "bert-base-uncased",
    "roberta-base",
    "albert-base-v2",
    "distilbert-base-uncased"
]

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [ ]:
# insert your code here (load dataset)

from datasets import load_dataset
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# dataset = load_dataset("truthfulqa/truthful_qa", 'multiple_choice')
dataset = load_dataset("truthfulqa/truthful_qa", 'generation')
dataset = dataset['validation'].filter(lambda x: len(x['correct_answers']) == 2) # make sure there are 2 correct answers
dataset = dataset.shuffle(seed=55).select(range(100))
dataset

In [ ]:
# insert your code here (load models for semantic similarity and QA)


In [ ]:
# insert your code here (function for optimal correct answers & semantic similarity)
import random
from transformers import AutoTokenizer, AutoModelForMultipleChoice


def get_predictions(model_name, model_names_semantic, dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name).to(device)
    model.eval()
    semantic_models = {
        name: SentenceTransformer(name).to(device)
        for name in model_names_semantic
    }
    correct_counts = {name: 0 for name in model_names_semantic}
    total = 0
    predictions = []

    for sentence in tqdm(dataset):
        labels = ["best", "correct1", "correct2", "incorrect"]
        question = sentence['question']
        options = [sentence['best_answer'], sentence['correct_answers'][0], sentence['correct_answers'][1], random.choice(sentence["incorrect_answers"])]
        
        # Shuffle options and update correct_idx accordingly
        labeled_options = list(zip(options, labels))
        random.shuffle(labeled_options)
        shuffled_answers, shuffled_labels = zip(*labeled_options)

        inputs = tokenizer(
            [question] * 4,
            list(shuffled_answers),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        ).to(device)

        input_reshape = {k: v.unsqueeze(0) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**input_reshape)
            logits = outputs.logits
            pred_idx = torch.argmax(logits, dim=1).item()

        # Get predicted answer and its original label
        predicted_answer = shuffled_answers[pred_idx]
        predicted_label = shuffled_labels[pred_idx]
        predictions.append({
            "question": sentence['question'],
            "chosen_answer": predicted_answer,
            "chosen_label": predicted_label,
            "all_options": list(shuffled_answers),
            "all_labels": list(shuffled_labels),
            "predicted_index": pred_idx
        })
        # Only check similarity if not incorrect
        if predicted_label != "incorrect":
            reference = sentence['best_answer']
            for model_s, semantic_model in semantic_models.items():
                pred_embed = semantic_model.encode(predicted_answer, convert_to_numpy=True)
                ref_embed = semantic_model.encode(reference, convert_to_numpy=True)
                similarity = get_cosine_similarity(pred_embed, ref_embed)
                if similarity >= 0.95:
                    correct_counts[model_s] += 1

        total += 1

    accuracy_per_model = {
        name: correct / total if total > 0 else 0.0
        for name, correct in correct_counts.items()
    }

    for name, acc in accuracy_per_model.items():
        print(f"Semantic model '{name}' accuracy: {acc:.2%}")

    return accuracy_per_model

def evaluate_multiple_models(model_names_B1, model_names_semantic, dataset):
    results = {}
    for mc_model in model_names_B1:
        print(f"\n🔍 Evaluating MC model: {mc_model}")
        acc = get_predictions(mc_model, model_names_semantic, dataset)
        results[mc_model] = acc
    return results

results = evaluate_multiple_models(model_names_B1, model_names_semantic, dataset)

In [ ]:
results

## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [48]:
# insert your code here (load dataset)
from datasets import load_dataset

dataset = load_dataset('allenai/winogrande', 'winogrande_xl', trust_remote_code=True)
dataset = dataset['validation'].shuffle(seed=55).select(range(100))
# dataset

Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [49]:
# insert your code here (load models)
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fill_mask = pipeline("fill-mask", model="roberta-large", device=device)
zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)



Device set to use cpu
Device set to use cpu


In [51]:
# insert your code here (create pipelines)
def evaluate_fill_mask(dataset):
    correct = 0
    for sample in dataset:
        sentence = sample["sentence"].replace("_", "<mask>")
        option1 = sample["option1"]
        option2 = sample["option2"]
        answer = int(sample["answer"])

        # Fill mask
        preds = fill_mask(sentence, top_k=10)
        tokens = [pred['token_str'].strip() for pred in preds]

        # Determine which option appears first in predictions
        if option1 in tokens and (option2 not in tokens or tokens.index(option1) < tokens.index(option2)):
            pred = 1
        elif option2 in tokens:
            pred = 2
        else:
            pred = 0  # Unknown

        if pred == answer:
            correct += 1

    acc = correct / len(dataset)
    print(f"[Fill-Mask] Accuracy: {acc:.2%}")
    return acc


def evaluate_deeppavlov(dataset):
    tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/roberta-large-winogrande")
    model = AutoModelForSequenceClassification.from_pretrained("DeepPavlov/roberta-large-winogrande").to(device)
    model.eval()

    correct = 0

    for sample in tqdm(dataset):
        sentence = sample["sentence"]
        option1 = sample["option1"]
        option2 = sample["option2"]
        answer = int(sample["answer"])

        # Split sentence around the blank "_"
        prefix, suffix = sentence.split("_")

        # Build sentence pairs
        sent1_1 = prefix.strip()
        sent2_1 = f"{option1.strip()} {suffix.strip()}"
        sent2_2 = f"{option2.strip()} {suffix.strip()}"

        # Tokenize and classify
        inputs1 = tokenizer(sent1_1, sent2_1, return_tensors="pt", padding=True, truncation=True).to(device)
        inputs2 = tokenizer(sent1_1, sent2_2, return_tensors="pt", padding=True, truncation=True).to(device)

        with torch.no_grad():
            logits1 = model(**inputs1).logits
            logits2 = model(**inputs2).logits

        # Output is [False_score, True_score]
        true_score1 = torch.softmax(logits1, dim=1)[0][1].item()
        true_score2 = torch.softmax(logits2, dim=1)[0][1].item()

        pred = 1 if true_score1 > true_score2 else 2

        if pred == answer:
            correct += 1

    acc = correct / len(dataset)
    print(f"[DeepPavlov (pairwise)] Accuracy: {acc:.2%}")
    return acc

def evaluate_zero_shot(dataset):
    correct = 0
    for sample in dataset:
        sentence = sample["sentence"].replace("_", "___")  # optional: make blank clearer
        candidate_labels = [sample["option1"], sample["option2"]]
        answer = int(sample["answer"])

        result = zero_shot(sentence, candidate_labels)
        pred = 1 if result["labels"][0] == sample["option1"] else 2

        if pred == answer:
            correct += 1

    acc = correct / len(dataset)
    print(f"[Zero-Shot] Accuracy: {acc:.2%}")
    return acc

In [52]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

def score_sentence(model, tokenizer, sentence):
    inputs = tokenizer(sentence, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
    return -outputs.loss.item()

def evaluate_lm(model_name, dataset):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    model.eval()

    correct = 0

    for sample in tqdm(dataset):
        s = sample["sentence"]
        s1 = s.replace("_", sample["option1"])
        s2 = s.replace("_", sample["option2"])
        label = int(sample["answer"])

        score1 = score_sentence(model, tokenizer, s1)
        score2 = score_sentence(model, tokenizer, s2)

        pred = 1 if score1 > score2 else 2
        if pred == label:
            correct += 1

    acc = correct / len(dataset)
    print(f"[Language Model - {model_name}] Accuracy: {acc:.2%}")
    return acc


In [53]:
# insert your code here (function for predicting best fill)

evaluate_fill_mask(dataset)
evaluate_deeppavlov(dataset)
evaluate_zero_shot(dataset)
evaluate_lm("gpt2", dataset)

[Fill-Mask] Accuracy: 52.00%


100%|██████████| 100/100 [00:11<00:00,  8.40it/s]


[DeepPavlov (pairwise)] Accuracy: 83.00%
[Zero-Shot] Accuracy: 50.00%


100%|██████████| 100/100 [00:07<00:00, 13.06it/s]

[Language Model - gpt2] Accuracy: 53.00%


0.53